In [1]:
import torch
print(torch.cuda.is_available())

True


In [2]:
!pip install transformers torch

In [4]:
import pandas as pd

file_path = "/kaggle/input/reviews-dataset/zero_shot_labeled_reviews_final.csv" 
df = pd.read_csv(file_path)

In [5]:
df.head()

,Username,Total reviews,Date,Review,Rating,Cleaned_Review,Tokenized_Review,Emotion
0,Richard,37K reviews,2025-02-14T22:43:05.000Z,Just when you thought Customer Disservice coul...,1,just when you thought customer disservice coul...,"[101, 2074, 2043, 2017, 2245, 8013, 4487, 1811...",sad
1,Ronald Mitzel,3.6K reviews,2025-02-14T06:03:30.000Z,Worst delivery of any store in the United Stat...,1,worst delivery any store united statesi paid m...,"[101, 5409, 6959, 2151, 3573, 2142, 2163, 2072...",frustrated
2,Armando Garcia Luna,14K reviews,2025-02-12T03:01:21.000Z,Poor customer service & ScamPoor customer serv...,1,poor customer service scampoor customer servic...,"[101, 3532, 8013, 2326, 8040, 16613, 16506, 80...",disappointed
3,Jessica Laurie,3reviews,2025-02-14T03:06:45.000Z,WalmartWalmart. Where do I even begin? It's ...,1,walmartwalmart where do i even begin its like ...,"[101, 24547, 22345, 13476, 22345, 2073, 2079, ...",frustrated
4,Mike Doyle,1review,2025-02-14T21:37:21.000Z,I bought a cellphone from an OSLI bought a cel...,1,i bought cellphone from an osli bought cellpho...,"[101, 1045, 4149, 3526, 9864, 2013, 2019, 9808...",disappointed


# **Aspect Extraction (for a sample)**

In [6]:
from transformers import pipeline

aspect_extractor = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


In [10]:
# Walmart service aspects based on my reviews dataset

aspect_labels = [
    "Customer Service",   # Issues related to support, responses, and resolutions
    "Delivery & Orders",  # Delivery speed, order issues, missing items
    "In-Store Experience",  # Cleanliness, waiting time, stock availability
    "Pricing & Billing",  # Pricing complaints, refunds, extra charges
]

In [16]:
# Run on a sample of 100 reviews (to test first)

sample_reviews = df["Review"].dropna().sample(100, random_state=42).tolist()

In [17]:
# Extract aspects using zero-shot classification

aspect_results = {review: aspect_extractor(review, aspect_labels, multi_label=True) for review in sample_reviews}

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [18]:
# Show a few results

for review, result in list(aspect_results.items())[:5]:
    print(f"Review: {review}")
    print(f"Predicted Aspects: {result['labels'][:3]}")  # Shows the top 3 predicted aspects
    print(f"Confidence Scores: {result['scores'][:3]}")
    print("-" * 80)

Review: Horrible on line experienceI ordered my smart fitness watch over a week ago first of all I don't understand why it takes so long to receive it when it's not that far away number two they said I would have it by the 20th it is now the 21st and I still don't have it and furthermore I checked and to see where it was the tracking on it and the post office still haven't received it. I will never ever ever again order anything online from Walmart I called customer service they were the least helpful.Date of experience:January 21, 2021
Predicted Aspects: ['Customer Service', 'Delivery & Orders', 'Pricing & Billing']
Confidence Scores: [0.9846453666687012, 0.8138330578804016, 0.19210220873355865]
--------------------------------------------------------------------------------
Review: AwesomeShipping takes forever but overall great companyDate of experience:January 21, 2017
Predicted Aspects: ['Delivery & Orders', 'Customer Service', 'Pricing & Billing']
Confidence Scores: [0.7163544297

# **Aspect Extraction (for the complete dataset)**

In [20]:
# Load Dataset

file_path = "/kaggle/input/reviews-dataset/zero_shot_labeled_reviews_final.csv"
df = pd.read_csv(file_path)

In [22]:
# Drop the missing values (if any)

df = df.dropna(subset=["Review"])

In [23]:
# Load Zero-Shot Classification Model (BART-MNLI)

aspect_extractor = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=0)  # Use GPU (cuda:0)

Device set to use cuda:0


In [24]:
aspect_labels = [
    "Customer Service", 
    "Delivery & Orders", 
    "In-Store Experience", 
    "Pricing & Billing"
]

In [26]:
# Run Aspect Extraction on all the Reviews

reviews = df["Review"].tolist()
aspect_results = [aspect_extractor(review, aspect_labels, multi_label=True) for review in reviews]

In [27]:
# Convert the Results to DataFrame

aspect_data = []
for review, result in zip(reviews, aspect_results):
    aspect_data.append({
        "review": review,
        "predicted_aspects": ", ".join(result["labels"][:3]),  # Top 3 predicted aspects
        "confidence_scores": ", ".join([str(round(score, 2)) for score in result["scores"][:3]])
    })

In [28]:
aspect_df = pd.DataFrame(aspect_data)

In [29]:
# Save the Processed Data

aspect_df.to_csv("/kaggle/working/aspect_extracted_reviews.csv", index=False)

In [30]:
# Show Sample Results

print(aspect_df.head())

                                              review  \
0  Just when you thought Customer Disservice coul...   
1  Worst delivery of any store in the United Stat...   
2  Poor customer service & ScamPoor customer serv...   
3  WalmartWalmart.  Where do I even begin?  It's ...   
4  I bought a cellphone from an OSLI bought a cel...   

                                   predicted_aspects confidence_scores  
0  Customer Service, In-Store Experience, Deliver...   0.87, 0.2, 0.15  
1  Customer Service, Delivery & Orders, In-Store ...  0.98, 0.77, 0.21  
2  Customer Service, Delivery & Orders, Pricing &...  0.89, 0.87, 0.63  
3  In-Store Experience, Pricing & Billing, Custom...   0.1, 0.04, 0.03  
4  Customer Service, Pricing & Billing, In-Store ...  0.58, 0.16, 0.16  


In [36]:
# Download CSV 

from IPython.display import FileLink
FileLink("/kaggle/working/aspect_extracted_reviews.csv")

/kaggle/working/aspect_extracted_reviews.csv

# **Sentiment Analysis on Extracted Aspects**

In [37]:
import pandas as pd
from transformers import pipeline

In [38]:
# Load the Extracted Aspects Dataset

file_path = "/kaggle/working/aspect_extracted_reviews.csv"  
df = pd.read_csv(file_path)

In [64]:
# Load Improved RoBERTa Sentiment Model

sentiment_analyzer = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest", device=0)  # Use latest model

tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


In [65]:
# Define Function to Split Long Reviews (Ensures No Chunk Exceeds 512 Tokens)

def split_review(text, max_length=400):
    words = text.split()
    chunks = []
    current_chunk = []

    for word in words:
        current_chunk.append(word)
        if len(" ".join(current_chunk)) > max_length:
            chunks.append(" ".join(current_chunk))
            current_chunk = []
    
    if current_chunk:
        chunks.append(" ".join(current_chunk))
    
    return chunks

In [66]:
# Define Function for Aspect-Based Sentiment Analysis
def analyze_aspect_sentiments(row, debug=False, max_debug_samples=5):
    if pd.isna(row["predicted_aspects"]):  # Handle missing aspect data
        return {}

    aspects = row["predicted_aspects"].split(", ")  # Convert aspect string to list
    aspect_sentiments = {}

    for aspect in aspects:
        review_text = f"{aspect}: {row['review']}"  # Create aspect-focused text
        review_chunks = split_review(review_text, max_length=400)

        chunk_sentiments = []
        for i, chunk in enumerate(review_chunks):
            chunk_tokens = tokenizer.encode(chunk)

            # Print debug info only for the first `max_debug_samples` reviews
            
            if debug and row.name < max_debug_samples:
                print(f"[DEBUG] Review {row.name}: Chunk {i+1} token length = {len(chunk_tokens)}")  

            if len(chunk_tokens) > 512:
                chunk = tokenizer.decode(chunk_tokens[:512])  # Trim to 512 tokens

            sentiment_result = sentiment_analyzer(chunk, truncation=True)[0]
            chunk_sentiments.append(sentiment_result["label"])

        # Majority vote for final sentiment
        
        final_sentiment = max(set(chunk_sentiments), key=chunk_sentiments.count)
        aspect_sentiments[aspect] = final_sentiment  

    return aspect_sentiments

In [68]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [70]:
sentiment_result = sentiment_analyzer(chunk, truncation=True).to("cpu")[0]

NameError: name 'chunk' is not defined

In [69]:
# Apply Sentiment Analysis to All Reviews

df["aspect_sentiments"] = df.apply(lambda row: analyze_aspect_sentiments(row, debug=True), axis=1)

[DEBUG] Review 0: Chunk 1 token length = 90


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
#hjjkdfvhmnfv

In [ ]:
# Save the Final Dataset with Sentiments

output_file = "/kaggle/working/aspect_sentiment_analysis.csv"
df.to_csv(output_file, index=False)

In [ ]:
# Display Sample Results

print(df.head())

In [ ]:
# Provide the Download Link for Output File

from IPython.display import FileLink
FileLink(output_file)